In [23]:
import sys
sys.path.insert(0, '/home/sos/Dialogue_Research/color_in_context/system/dialogue_manager')
from dataset.cic import make_or_load_cic
import dataset
from chart_parser.AStarQ import AStarQ
from chart_parser.node import node

from chart_parser.util import load_grammar
from chart_parser.util import load_grammar_intent_map
from chart_parser.logic import get_logical_form
import chart_parser
import os
import pickle

In [24]:
def initialize_state(start_k, first_variable, state_tracker, grammar):
    start_state, prob = grammar[first_variable][0]
    start_state = (start_state, (start_k, 0), first_variable, start_k, -1, 0)
    
    start_node = node(first_variable, [], None, start=start_k, end=-1)
    state_tracker[start_k][(start_state, (-start_k, -prob))] = start_node
    
    queue = AStarQ()
    queue.add_node(start_state, start_node, (-start_k, -prob), 0)
    
    max_node = (start_node, (-start_k, -prob))
    
    return queue, state_tracker, max_node

In [25]:

#state representation
#(rule, (sentence_ix_where_match_started, dot_ix), parent_var, sentence_ix_being_matched,\
#parent_ix_in_the_rule_being_matched, my_index_in_the_rule_being_matched)
# global ignored_count
# global total_count
# ignored_count = 0
# total_count = 0

def make_copy(parse_tree):
    node_copy = node(parse_tree.var, [], parse_tree.parent, start=parse_tree.start, end=parse_tree.end)
    for child in parse_tree.children:
        node_copy.children.append(make_copy(child))
    return node_copy

def check_if_in_first(variable, nonterminals, terminals, first_set, k, string):
#     if k < len(string) and variable not in nonterminals:
#         variations = set([string[k] + x for x in ['er', 'ish', 'est', 'der', 'dish', '']])
#         if variable not in variations:
#             return True
    if k < len(string) and (variable in nonterminals) and \
        (string[k] not in first_set[variable]):
        return True
    return False

def get_init_node(start_index, first_variable, grammar):
    pass

def chart_parse(string, grammar, terminals, nonterminals, first_set, first_variable):
    print('Received String:', string)
    
    queue = None
    state_tracker = [dict() for k in range(len(string) + 1)]
    max_node = tuple()
    partial_trees = list()

    def predictor(state, k, grammar):
        #modify this predictor to look at the string
        if k < len(string) and string[k] not in terminals:
#             print('THIS IS WHERE EVERYTHING WENT WRONG!')
            return
        elif k >= len(string):
            return
        rule, (i, dot_ix), parent_var, k, prix, rix = state.state
        parent_node = state.node
        
        rule = rule.split()

        if check_if_in_first(rule[dot_ix], nonterminals, terminals, first_set, k, string):
            return

        cur_cost = state.cost
        hrst = 0
        for ri, (next_rule, prob) in enumerate(grammar[rule[dot_ix]]):

            left_most = next_rule.split()[0].strip()
            if check_if_in_first(left_most, nonterminals, terminals, first_set, k, string):
                continue

            next_state = (next_rule, (k, 0), rule[dot_ix], k, rix, ri)
            child_node = node(rule[dot_ix], [], parent_node, start=k)
            child_cost = (-k, cur_cost[1] * prob)

            state_tracker[k][(next_state, child_cost)] = child_node #q_entry.node
            queue.add_node(next_state, child_node, child_cost, hrst)
        

    def scanner(state, k, max_node):
        rule, (i, dot_ix), parent_var, k, prix, rix = state.state
        parent_node = state.node

        split_rule = rule.split()

        cur_cost = state.cost
        hrst = 0
        if k >= len(string):
            return max_node
        variations = set([split_rule[dot_ix] + x for x in ['er', 'ish', 'est', 'der', 'dish', '']])
#         if 'pink' in split_rule[dot_ix]:
#             print(state.state)
#             print(variations)
#         print('IN SCANNER:', parent_var, rule)

        if string[k] in variations:
            next_state = (rule, (i, dot_ix + 1), parent_var, k + 1, prix, rix)
            
            parent_node.children.append( node(split_rule[dot_ix], [], parent_node, \
                start=k, end=k+1) )
            parent_node.end = max(k + 1, parent_node.end)
            
            if parent_node and parent_node.end > max_node[0].end:
                max_node = (parent_node, (-k-1, cur_cost[1]))
            elif parent_node and parent_node.end == max_node[0].end and (-k-1, cur_cost[1]) < max_node[1]:
                max_node = (parent_node, (-k-1, cur_cost[1]))
            
            if dot_ix + 1 < len(split_rule):
                if check_if_in_first(split_rule[dot_ix + 1], nonterminals, terminals,\
                    first_set, k + 1, string):
                    return max_node
#             print('I am here', next_state)

            queue.add_node(next_state, parent_node, (-k-1, cur_cost[1]), hrst)
        return max_node

    def completer(state, k, max_node):
        rule, (i, dot_ix), parent_var, state_k, pprix, prix = state.state
        complete_node = state.node

        cost = state.cost
        hrst = 0
        
        for (entry,cur_cost) in list(state_tracker[i].keys()):
            cur_rule, (cur_i, cur_dot), cur_parent, cur_k, _, rix = entry

            cur_node = state_tracker[i][(entry,cur_cost)]

            split_cur_rule = cur_rule.split()

            if cur_dot >= len(split_cur_rule):
                continue

            if parent_var == split_cur_rule[cur_dot]:
                
                if (cur_dot+1) < len(split_cur_rule):
                    if check_if_in_first(split_cur_rule[cur_dot + 1], nonterminals, terminals,\
                        first_set, k, string):
                        continue
                
                cur_node = make_copy(state_tracker[i][(entry,cur_cost)])
                next_state = (cur_rule, (cur_i, cur_dot + 1), cur_parent, k, rix, prix)

                cur_node.children.append(complete_node)
                cur_node.end = complete_node.end
                complete_node.parent = cur_node

                if cur_parent == '<P>' and cur_node.end == max_node[0].end and \
                    (-k, cur_cost[1]) <= max_node[1]:
                    max_node = (cur_node, (-k, cost[1]))
                state_tracker[k][(next_state, (-k, cost[1]))] = cur_node
                queue.add_node(next_state, cur_node, (-k, cost[1]), hrst)
        return max_node

    start_k = 0
#     queue, state_tracker, max_node = initialize_state(0, first_variable, state_tracker, grammar)
    
    start_state, prob = grammar[first_variable][0]
    start_state = (start_state, (0, 0), first_variable, 0, -1, 0)
    queue = AStarQ()
    start_node = node(first_variable, [], None, start=0, end=-1)
    state_tracker[0][(start_state, (0, -prob))] = start_node
    queue.add_node(start_state, start_node, (0, -prob), 0)
    max_node = (start_node, (0, -prob))
    
    #####
    # global total_count
    while True:
        try:
            q_entry = queue.pop_node()
        except KeyError as e:
            print('cannot be parsed', e, max_node[0], (max_node[0].start, max_node[0].end))
            start_k = -1

            if max_node[0].end <= 0:
                partial_trees.append(None)
                start_k = max_node[0].start + 1
            elif max_node[0].var != '<P>':
                partial_trees.append(None)
                start_k = max_node[0].end
            else:
                partial_trees.append(max_node[0])
                start_k = max_node[0].end

            while (start_k < len(string)) and (string[start_k] not in first_set[first_variable]):
                start_k += 1
            
            print(start_k)
            
            if start_k >= len(string):
                return partial_trees

            state_tracker = [dict() for k in range(len(string) + 1)]
            queue, state_tracker, max_node = initialize_state(start_k, first_variable, state_tracker, grammar)
            q_entry = queue.pop_node()

        rule, (i, dot_ix), parent_var, k, _, rix = q_entry.state
#         print(rule, i, dot_ix, parent_var, k)

        rule = rule.split()
        cur_cost = q_entry.cost

        if (q_entry.state, cur_cost) not in state_tracker[k]:
            state_tracker[k][(q_entry.state, cur_cost)] = q_entry.node

        if q_entry.node.parent == None and \
            q_entry.node.start <= start_k and q_entry.node.end >= len(string):
            return partial_trees + [q_entry.node]

        if not (dot_ix == len(rule)):
            if rule[dot_ix] in nonterminals:
                predictor(q_entry, k, grammar)
            else:
                max_node = scanner(q_entry, k, max_node)
        else:
            max_node = completer(q_entry, k, max_node)

In [28]:
def tokenize(string):
    symbol_list = ['.', ',', '?', '/', '\\', '?', '-', ';', '!']
    utterance = string.lower()
    for s in symbol_list:
        utterance = (' ' + s + ' ').join(utterance.split(s)).strip()
    return utterance.split()

In [29]:
test_sentences = [
    'dark pink',
    'dark blue',
    'mauve and dark blue',
    'mauve, not dark blue not hot pink'
]

def create_level_dict(vocabulary):
    level_dict = dict()
    for i in range(len(vocabulary)):
        color_term = vocabulary.lookup_index(i)
        term_tokens = color_term.split()
        for j in range(len(term_tokens)):
            level_dict.setdefault(j, dict())
            level_dict[j].setdefault(term_tokens[j], set())
            level_dict[j][term_tokens[j]].add(i)
    return level_dict

def preprocess(sentence, vocabulary):
    start = 0
    result = []
    while start < len(sentence):
        max_length = 0
        term_ix = -1
        for i in range(0, 3):
            cur_term = ' '.join(sentence[start:start + i + 1])
            try:
                term_ix = vocabulary[cur_term]
                max_length = i + 1
            except Exception as e:
                pass
        if max_length == 0:
            result.append(sentence[start])
            start += 1
        else:
            result.append(str(term_ix))
            start += max_length
    return result
            
if __name__ == '__main__':
    cic = make_or_load_cic()
    cic._color_vocab = pickle.load(open(os.path.dirname(dataset.__file__) + '/extended_vocabulary.pkl', 'rb'))
    dir_name = os.path.dirname(chart_parser.__file__)
    g, t, n, fs = load_grammar(cic._color_vocab, dir_name + '/syntaxparsingrules_V3')
    print(g['<CLR>'])
    g = pickle.load(open(os.path.dirname(dataset.__file__) + '/pcfg_3.pkl', 'rb'))
    print(g['<S>'])
    print(max([x[1] for x in g['<S>']]))
    irmap, rimap = load_grammar_intent_map(dir_name + '/gram_intent_map')
#     extended_vocab
#     print(irmap)
#     print(rimap)

#     print('and' in fs['<S>'],'and' in fs['<ConjP>'], 'and' in fs['<conj>'])
#     print('Length of Terminals:', len(t), 'Terminals:', t)

    for sentence in test_sentences:
        print('To Parse:', sentence)
        sentence = tokenize(sentence)
        sentence = preprocess(sentence, cic._color_vocab)
        parse_tree = chart_parse(sentence, g, t, n, fs, '<P>')
        for p in parse_tree:
            if p:     
                p.print_tree()
                print('Logical Form:', get_logical_form(cic._color_vocab, rimap, irmap, p, 'E'))
        print('-----------------')
    print('We are done!')

keys: <P>
keys: <S>
keys: <CP>
keys: <adj>
keys: <N>
keys: <Q>
keys: <det>
keys: <ConjP>
keys: <conj>
keys: <DescP>
keys: <NegP>
keys: <neg>
keys: <confirmation>
keys: <denial>
keys: <emoji>
keys: <hmmm>
[('0', 0.0012062726176115801), ('1', 0.0012062726176115801), ('2', 0.0012062726176115801), ('3', 0.0012062726176115801), ('4', 0.0012062726176115801), ('5', 0.0012062726176115801), ('6', 0.0012062726176115801), ('7', 0.0012062726176115801), ('8', 0.0012062726176115801), ('9', 0.0012062726176115801), ('10', 0.0012062726176115801), ('11', 0.0012062726176115801), ('12', 0.0012062726176115801), ('13', 0.0012062726176115801), ('14', 0.0012062726176115801), ('15', 0.0012062726176115801), ('16', 0.0012062726176115801), ('17', 0.0012062726176115801), ('18', 0.0012062726176115801), ('19', 0.0012062726176115801), ('20', 0.0012062726176115801), ('21', 0.0012062726176115801), ('22', 0.0012062726176115801), ('23', 0.0012062726176115801), ('24', 0.0012062726176115801), ('25', 0.0012062726176115801),

<S> 0 3
       [<P>[0,3]]       
       [<S>[0,3]]       
    [<S>[0,1]    <conj>[1,2]    <S>[2,3]]    
    [<CP>[0,1]]    [and[1,2]]    [<CP>[2,3]]    
    [<CLR>[0,1]]    []    [<CLR>[2,3]]    
     [486[0,1]]     [165[2,3]]     
     []     []     
Children Right Now: ['<S>', '<conj>', '<S>']
and 1
and 2
Some exception occured: substring not found
This is the split Rule ['Rule', 'COMPARE_REF(T,[and],A,B)'] 1
Children Right Now: ['<CLR>']
486 0
486 1
Some exception occured: substring not found
This is the split Rule ['Rule', 'IDENTIFY(A,486)'] 1
Going to split IDENTIFY(A,486)
Split arguments: IDENTIFY A,486)
Children Right Now: ['<CLR>']
165 0
165 1
Some exception occured: substring not found
This is the split Rule ['Rule', 'IDENTIFY(A,165)'] 1
Going to split IDENTIFY(A,165)
Split arguments: IDENTIFY A,165)
Logical Form: ('COMPARE_REF(T,[and],A,B) AND IDENTIFY(A,486) AND IDENTIFY(B,165)', 'E')
-----------------
To Parse: mauve, not dark blue not hot pink
Received String: ['486', ',',

In [8]:
max_length = 0
for i in range(len(cic._color_vocab)):
    color_term = cic._color_vocab.lookup_index(i).split()
    max_length = max(max_length, len(color_term))
# print(max_length)
# print(t)

In [9]:
# x = ['1', '2', '3']
# variations = ['a', 'b', 'c', 'd', 'c', 'e', '']

# def all_variations(x, variations):
#     if len(x) == 1:
#         return [[x[0] + var] for var in variations]
#     to_merge = all_variations(x[1:], variations)
#     result = []
#     for entity in to_merge:
#         for var in variations:
#             to_append = list(entity)
#             to_append.insert(0, x[0] + var)
#             result.append(to_append)
#     return result
# all_variations(x, variations)

[['1a', '2a', '3a'],
 ['1b', '2a', '3a'],
 ['1c', '2a', '3a'],
 ['1d', '2a', '3a'],
 ['1c', '2a', '3a'],
 ['1e', '2a', '3a'],
 ['1', '2a', '3a'],
 ['1a', '2b', '3a'],
 ['1b', '2b', '3a'],
 ['1c', '2b', '3a'],
 ['1d', '2b', '3a'],
 ['1c', '2b', '3a'],
 ['1e', '2b', '3a'],
 ['1', '2b', '3a'],
 ['1a', '2c', '3a'],
 ['1b', '2c', '3a'],
 ['1c', '2c', '3a'],
 ['1d', '2c', '3a'],
 ['1c', '2c', '3a'],
 ['1e', '2c', '3a'],
 ['1', '2c', '3a'],
 ['1a', '2d', '3a'],
 ['1b', '2d', '3a'],
 ['1c', '2d', '3a'],
 ['1d', '2d', '3a'],
 ['1c', '2d', '3a'],
 ['1e', '2d', '3a'],
 ['1', '2d', '3a'],
 ['1a', '2c', '3a'],
 ['1b', '2c', '3a'],
 ['1c', '2c', '3a'],
 ['1d', '2c', '3a'],
 ['1c', '2c', '3a'],
 ['1e', '2c', '3a'],
 ['1', '2c', '3a'],
 ['1a', '2e', '3a'],
 ['1b', '2e', '3a'],
 ['1c', '2e', '3a'],
 ['1d', '2e', '3a'],
 ['1c', '2e', '3a'],
 ['1e', '2e', '3a'],
 ['1', '2e', '3a'],
 ['1a', '2', '3a'],
 ['1b', '2', '3a'],
 ['1c', '2', '3a'],
 ['1d', '2', '3a'],
 ['1c', '2', '3a'],
 ['1e', '2', '3a'],
 ['1

In [19]:
class ExtendedVocabulary():
    def __init__(self):
        self._vocab = pickle.load(open(os.path.dirname(dataset.__file__) + '/vocabulary.pkl', 'rb'))
        self._extended = dict()
        self._variations = ['er', 'ish', 'est', 'der', 'dish', '']
        self.extend()
    
    def all_variations(self, x, variations):
        if len(x) == 1:
            return [[x[0] + var] for var in variations]
        to_merge = all_variations(x[1:], variations)
        result = []
        for entity in to_merge:
            for var in variations:
                to_append = list(entity)
                to_append.insert(0, x[0] + var)
                result.append(to_append)
        return result

        
    def extend(self):
        for i in range(len(self._vocab)):
            term_tokens = self._vocab.lookup_index(i).split()
            print('These are tokens for color terms:', term_tokens)
            variations = self.all_variations(term_tokens, self._variations)
            for var in variations:
                new_cterm = ' '.join(var)
#                 print(new_cterm)
                self._extended[new_cterm] = i
#             break
                
    def __getitem__(self, token):
        if token not in self._extended:
            raise KeyError('Implicit adds not supported')
        return self._extended[token]
    
    def lookup_index(self, index):
        if index >= len(self._vocab):
            raise KeyError('the index (%d) is not in the vocabulary' % index)
        return self._vocab.lookup_index()
    
    def __len__(self):
        return len(self._vocab)
# extended_vocab = ExtendedVocabulary()
# #inclusion test
# # all([cic._color_vocab.lookup_index(i) in extended_vocab._extended for i in range(len(cic._color_vocab))])
# pickle.dump(extended_vocab, open(os.path.dirname(dataset.__file__) + '/extended_vocabulary.pkl', 'wb'))
# len(extended_vocab._extended.keys())

These are tokens for color terms: ['acid', 'green']
These are tokens for color terms: ['adobe']
These are tokens for color terms: ['algae']
These are tokens for color terms: ['algae', 'green']
These are tokens for color terms: ['almost', 'black']
These are tokens for color terms: ['amber']
These are tokens for color terms: ['amethyst']
These are tokens for color terms: ['apple']
These are tokens for color terms: ['apple', 'green']
These are tokens for color terms: ['apricot']
These are tokens for color terms: ['aqua']
These are tokens for color terms: ['aqua', 'blue']
These are tokens for color terms: ['aqua', 'green']
These are tokens for color terms: ['aquamarine']
These are tokens for color terms: ['army', 'green']
These are tokens for color terms: ['asparagus']
These are tokens for color terms: ['aubergine']
These are tokens for color terms: ['auburn']
These are tokens for color terms: ['avocado']
These are tokens for color terms: ['avocado', 'green']
These are tokens for color ter

34792